In [60]:
import random

import gym
import gymnasium
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Dropout
from collections import deque

In [61]:
env=gym.make('MountainCar-v0',render_mode='human')

In [62]:
env.reset( )

(array([-0.40304565,  0.        ], dtype=float32), {})

In [63]:
env.render()

In [64]:
env.observation_space.shape[0]

2

In [65]:
env.action_space.n


3

In [66]:
env.action_space.sample()

0

In [67]:
class DQL:
    def __init__(self,env):
        self.env=env
        self.memory=deque(maxlen=2000)
        self.gamma=0.03
        self.epsilon=1.0
        self.epsilon_min=0.01
        self.epsilon_decay=0.995
        self.learning_rate=0.005
        self.tau=0.125
        self.model=self.create_model()
        self.target_model=self.create_model()
        def create_model(self):
            model=Sequential()
            model.add(Dense(24,input_dim=self.env.observation_space.shape[0],activation='relu'))
            model.add(Dense(48,activation='relu'))
            model.add(Dense(24,activation='relu'))
            model.add(self.env.action_space.n)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=self.learning_rate))
            return model
        def act(self,state):
            self.epsilon*=self.epsilon_dacay
            self.epsilon=max(self.epsilon_min, self.epsilon)
            if np.random.random()<self.epsilon:
                return  self.env.action_space.sample()
            else:
                return np.argmax(self.model.predict(state))
        def replay(self,state):
            batch_size=32
            if len(self.memory)<batch_size:
                return
            else:
                samples=random.sample(self.memory,batch_size)
                for sample in samples:
                    state,action,reward,new_state,done=sample
                    target=self.target_model.predict(state)
                    if done:
                        target[0][action]=reward
                    else:
                        Q_future=max(self.target_model.predict(new_state)[0])
                        target[0][action]=reward*Q_future+self.gamma
                    self.model.fit(state,target,epoch=1,verbose=1)
